In [3]:
import tensorflow
from tensorflow import keras
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D
from keras import Sequential

###### images ka data bht bara hai aur aik sath data ko RAM mein load krna impossible ho jata hai kyu k RAM chota parr jata hai is cheez ko handle krne k lye generators ka use kya jata hai 

###### generators -- > ye data ko choty choty batches mein divide krta hai aur aik batch aik time mein RAM mein load hota hai jb us ka kaam khtm ho jata hai to wo RAM se hatt jata hai aur agla batch load hota hai.

# Genarators

In [8]:
# training dataset
train_ds = keras.utils.image_dataset_from_directory(
    directory = 'catsvsdogs/train', # path of directory of training data
    labels = 'inferred',
    label_mode = 'int', # it will assign 1 to dogs and 0 to cats
    batch_size = 32,
    image_size = (256,256)
)

# testing data
validation_ds = keras.utils.image_dataset_from_directory(
    directory = 'catsvsdogs/test', # path of directory of training data
    labels = 'inferred',
    label_mode = 'int', # it will assign 1 to dogs and 0 to cats
    batch_size = 32,
    image_size = (256,256)
)
    

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


###### It will read data in the form of numpy array 

###### numpy arrays mein 0-255 values aye hain is lye is ko normalize krne ki zroort hai

# Normalization

In [11]:
def process(image,label):
    image = tensorflow.cast(image/255 , tensorflow.float32) # pixel ki hrr value ko 255 se divide kre ga aur us ko float mein rkhe ga
    return image,label
train_ds = train_ds.map(process) 
validation_ds = validation_ds.map(process)

# Creating CNN model

In [14]:
model = Sequential()

model.add(Conv2D(32,kernel_size = (3,3) ,padding = 'valid',activation = 'relu' , input_shape=(256,256,3)))
model.add(MaxPooling2D(pool_size = (2,2) ,strides = 2,padding = 'valid'))

model.add(Conv2D(64,kernel_size = (3,3) ,padding = 'valid',activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) ,strides = 2,padding = 'valid'))

model.add(Conv2D(128,kernel_size = (3,3) ,padding = 'valid',activation = 'relu' ))
model.add(MaxPooling2D(pool_size = (2,2) ,strides = 2,padding = 'valid'))

model.add(Flatten())

model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 254, 254, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 127, 127, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 125, 125, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 62, 62, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 60, 60, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 30, 30, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 115200)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │      14,745,728 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,847,297 (56.64 MB)

 Trainable params: 14,847,297 (56.64 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [ ]:
history = model.fit(train_ds,epochs = 10,validation_data = validation_ds)